<a href="https://colab.research.google.com/github/F423058/socu-ids-prg01-2024-ex01--F423058-/blob/main/ex13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第13回課題

In [49]:
import os

# 自動採点を邪魔せずにテストを実行するための関数
def local_run(f):
    if not ('GRADING' in os.environ):
        return f()

## Q1.

山陽小野田市民は選挙と開票が大好きなことで有名である．今日は選挙管理委員会の次期委員長を決める選挙の日である．今，投票が終わり，開票が始まろうとしている．山陽小野田市民は，開票中に少しでも早く当選者を知りたいと強く望んでいる．

当然のことながら, もっとも多くの票を得た候補者が次期委員長に選出される．たとえば，候補者が A, B, C の 3 人で，投票数が 10 票の場合を考える．そして，10 票のうち 6 票までの開票が行われ，A, B, C がそれぞれ 4 票，1 票，1 票を得たとする．この時点では，すべての候補者があと 4 票を得る可能性があり，全員が当選する可能性を残している．しかし，もし次に開いた票が A に投じられたものなら，A の当選が確定する．なぜならば，ここまでの A の得票は 5 票となり，最終的な B と C それぞれの得票は，高々 4 票にしかならないからである．この例では，7 票目を開票した時点で，山陽小野田市民は当選者を知ることができる．

そこで，投票データファイル名を受け取り，当選者が誰であるかとそれがいつ確定したかを求める関数`get_winner()`を作成してほしい．

### 投票データファイルの形式

投票データファイルは最大で1500個のデータセットからなる． 各データセットは2行からなり次のような形式である．

$\begin{array}{l}
n\\
c_1\ c_2\ \cdots\ c_n
\end{array}$

1行目の整数$n$は投票数を表し，100以下の正整数である． 2行目が投票された$n$票を表す． 各 $c_i（1 \leq i \leq n）$は，一つの英大文字（'A' から 'Z' のうちの一つ）であり，空白文字で区切られる．これは$i$番目の票が投じられた候補者を表す． 開票は，$c_1$ から $c_n$ まで出現順に行われる．

たとえすべての票が一人の候補者に集中していても，候補者は二人以上いると仮定すること．
またデータの終わりは，1個のゼロだけからなる行で表される．

以下のsample_eq1.txtは投票データファイルの例である. (一行目の`%%writefile sample_q1.txt`は無視せよ)

In [6]:
%%writefile sample_q1.txt
1
A
4
A A B B
5
L M N L N
6
K K K K K K
6
X X X Y Z X
10
A A A B A C A C C B
10
U U U U U V V W W W
0

Overwriting sample_q1.txt


### `get_winner()`の返値

各データセットについて以下を要素とするリストを返せ
- 選挙が引き分けに終わらない(当選者が存在する)なら，英大文字$c$と整数$d$からなるタプル$(c, d)$を返せ． ここで，$c$は当選者を表す．$d$は何番目の票を開票した時点ではじめて当選が確定したかを表す．
- それ以外の場合，すなわち選挙が引き分けに終わった場合は，`TIE' のみからなる文字列．

例えば上記のsample_q1.txtに対する返値は以下のリストである.

```python
[('A', 1), 'TIE', 'TIE', ('K', 4), ('X', 5), ('A', 7), ('U', 8)]
```

### Step 1. `get_winner()`をテストする`test_get_winner()`を完成

In [54]:
def test_get_winner():
    """
    test_winner()をテストする
    """
    file_name = 'sample_q1.txt'
    expected_result = [('A', 1), 'TIE', 'TIE', ('K', 4), ('X', 5), ('A', 7), ('U', 8)]
    result = get_winner(file_name)

    assert result == expected_result

### Step 2. `get_winner()`を完成

In [53]:
def get_winner(file_name):
    """
    与えられたファイル名のファイルに記入された投票データから、当選者と開票数を取得する

    Parameters:
        file_name (str): 投票データファイルの名前

    Returns:
        list: 当選者と当選が確実となった開票数のタプルのリスト
    """
    results = []

    # ファイルを開いて全ての行を読み込む
    with open(file_name, 'r') as file:
        data = file.read().strip().split('\n')

    i = 0
    while i < len(data):
        # 投票数を読み取る
        n = int(data[i])

        # 投票数がゼロの場合、ファイルの終わり
        if n == 0:
            break

        # 投票データを読み取る
        votes = data[i + 1].split()
        i += 2

        # 投票数が指定された数と一致しない場合は、データセットをスキップ
        if len(votes) != n:
            continue

        total_votes = len(votes)
        half_votes = total_votes / 2  # 総投票数の半分を計算
        vote_counts = {}  # 各候補者の投票数を記録する辞書
        winner_found = False

        # 各投票を処理する
        for idx, vote in enumerate(votes):
            # 候補者の投票数をインクリメント
            if vote in vote_counts:
                vote_counts[vote] += 1
            else:
                vote_counts[vote] = 1

            # 現在の候補者が総投票数の半分以上の票を得ているか確認
            if vote_counts[vote] > half_votes:
                # 当選者とその票が確定する時点を記録
                results.append((vote, idx + 1))
                winner_found = True
                break

        # 当選者が見つからなかった場合、引き分け
        if not winner_found:
            results.append('TIE')

    return results

### Step 3. テストを実行

In [55]:
local_run(test_get_winner)

AssertionError: 

## Q2.

君は小さな海賊団の一員である(ひとつなぎの大秘宝を目指しているかどうかは知らない). 海賊団は何枚かの地図を持っており, いずれの地図も, 同じ大きさの正方形に区切られたメッシュ状になっている．この地図は，ある海域を表しており，各正方形の領域が陸または海に対応する．以下は地図の一例である．

![](https://judge.u-aizu.ac.jp/onlinejudge/IMAGE1/2009B2.png)

陸に対応する二つの正方形領域が，地図上で縦，横または斜め方向に隣接しているなら，一方から他方へ歩いて行くことができる．陸に対応する二つの領域が同じ島に属するのは，一方から他方へ（一般には別の陸地を経由して）歩いて行ける時であり，またその時のみである．なお，この地図の海域は海で囲まれており，その外側へ歩いて出ることはできない．

君の任務は地図データファイル名を受け取り，地図データを解析してこの海域に島がいくつあるか数える関数`count_islands()`を作成することである．たとえば，上の図で表される地図には全部で三つの島がある．

### 地図データファイルの形式

地図データファイルには複数のデータセットが記入されており，各データセットは次のような形式で与えられる．

$\begin{array}{cccc}
w & h &  & \\
c_{1,1} & c_{1,2} & \cdots &  c_{1,w}\\
c_{2,1} & c_{2,2} & \cdots & c_{2,w}\\
\vdots & \vdots & \ddots & \vdots \\
c_{h,1} & c_{h,2} & \cdots & c_{h,w}
\end{array}$

- $w$と$h$は地図の幅と高さを表す50以下の正の整数であり，地図は$w\times h$個の同じ大きさの正方形から構成される．$w$と$h$の間は空白文字1個で区切られる．
- $c_{i,j}$は0または1であり，空白文字1個で区切られる. $c_{i, j}=0$なら，地図上で左から$i$番目，上から$j$番目の正方形は海であり，$c_{i, j} = 1$なら陸である．
- 入力の終わりは，空白文字1個で区切られた2個のゼロのみからなる行で表される．

以下のsample_q2.txtは地図データファイルの例である。(一行目の`%%writefile sample_q1.txt`は無視せよ)

In [ ]:
%%writefile sample_q2.txt
1 1
0
2 2
0 1
1 0
3 2
1 1 1
1 1 1
5 4
1 0 1 0 0
1 0 0 0 0
1 0 1 0 1
1 0 0 1 0
5 4
1 1 1 0 1
1 0 1 0 1
1 0 1 0 1
1 0 1 1 1
5 5
1 0 1 0 1
0 0 0 0 0
1 0 1 0 1
0 0 0 0 0
1 0 1 0 1
0 0

### `count_islands()`の返値

各データセットに対する島の個数からなるリストを返せ. 例えば上記のsample_q2.txtに対する返値は以下のリストである.

```python
[0, 1, 1, 3, 1, 9]
```

### Step 1. `count_islands()`をテストする`test_count_islands()`を完成

In [ ]:
def test_count_islands():
    """
    count_islands()をテストする
    """

### Step 2. `count_islands()`を完成

In [ ]:
def count_islands(file_name):
    """
    与えられたファイル名のファイルに記入された地図データを読み込み、島の個数を数える

    Parameters:
        file_name (str): 地図データファイルの名前

    Returns:
        list: 島の個数からなるリスト
    """

### Step 3. テストの実行

In [ ]:
local_run(test_count_islands)